# Sysmon - Review last 24 hours of execution

This notebook is intented to identify malicious or unknown applications that have been executed network wide in the past 24 (adjustable) hours. 

### Package install. 

This may not need to be run every time, just needed on environment creation

In [ ]:
#%%capture
%pip install --upgrade opensearch-py 

### Imports

Import the required packages

In [1]:
#Perform the necessary imports.
from opensearchpy import OpenSearch
from opensearchpy.helpers import scan

#Date
from datetime import datetime, timedelta

#Password 
from getpass import getpass

#JSON
import json

#Requests
import requests

### User Variables

In [25]:
duration = 24 #Duration to go back in hours
include_signed = False #If set to True, will process all applications, if false, will only searched unsigned apps
include_path_mismatch = True #If set to True, will print path mismatch with VT, if false, will exclude this check

### System Variables

Do not modify the block below unless you know what you are doing!

In [8]:
hosts = [{"host": "os01", "port": 9200}] #Opensearch details
username = 'admin' #Username, there is always an admin account in DFIR2Go
password = getpass(prompt="OpenSearch password for {user}:".format(user=username)) #Prompt the user for the password 
auth = (username, password) #Create the authentication details

ca_certs_path = '/certs/web/user-certs/ca/ca.crt' #Certs path

#Index
index = 'artifact_windows_sysinternals_sysmonlogforward*'

#VirusTotal API Key
vt_api_key = getpass(prompt="VirusTotal API Key:") #Prompt VT API Key

### Connection to OpenSearch

In [9]:
client = OpenSearch(
    hosts=hosts, 
    http_compress=True,
    http_auth=auth,
    use_ssl=True,
    timeout=300,
    verify_certs=True,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    ca_certs=ca_certs_path        
)


### Search the database for recent results

This section will define the query and interact with OpenSearch to obtain the results

In [26]:
#Define the query
query = {
    "size": "10000",  
    "timeout": "300s",
    "query": {
        "bool": {
            "must": [],
            "filter": [
              {
                "match": {
                  "ID": 1
                }
              },
              {
                "range": {
                  "Timestamp": {
                    "gte": datetime.now() - timedelta(hours=duration),
                    "lte": datetime.now(),
                    "format": "strict_date_optional_time"
                  }
                }
              }
            ],
            "should": [],
            "must_not": []
          }
    }
}

#Clear results
processes = []
for results in scan(client, query=query, index=index):
    t_proc = {} #Clear the temp dictionary

    #Filter results
    if include_signed or not include_signed and results['_source']['EventData']['Company'] == '-':
      #Now get a list of unique MD5 hashes  
      try:
          #Extract the MD5 hash
          tmp_hashes = results['_source']['EventData']['Hashes'].split(',')
          t_proc['md5_hash'] = tmp_hashes[0].replace("MD5=","")

          #Get the image path. 
          t_proc['path'] = results['_source']['EventData']['Image']

          #Get the hostname
          t_proc['host'] = results['_source']['Hostname']

          #Check if the hash is in the list
          if t_proc not in processes:
              processes.append(t_proc)
              #print('Added Hash for ' + results['_source']['EventData']['Image'])
              #print(results['_source']['EventData']['CommandLine'])
          #else:
              #print('Hash not added for ' + results['_source']['EventData']['Image'])
                  
      except:
          print("Command Line Missing")


### Enrich results

Search VirusTotal for each of the hashes looking for anything ususual. 

In [ ]:
for proc in processes:
    #Define the variables
    hash = proc['md5_hash']
    path = proc['path']
    host = proc['host']
    print_results = False #Will we display anything for this result
    result = '' #Blank Result variable

    result += '\nReviewing: {path} ({hash}) from {host}'.format(path=path, hash=hash, host=host)

    vtdata = requests.get("https://www.virustotal.com/api/v3/files/{hash}".format(hash=hash), headers={"x-apikey":vt_api_key})

    if vtdata.status_code == 200:
        try:
            vtjson = vtdata.json() #Convert to JSON
            
            #Work out if there is a file or not
            if 'error' in vtjson:
                print_results = True #Set the flag to true
                result = '\n[ERROR]: {message}'.format(message=jtson['error']['message'])
            elif 'data' in vtjson:              
                #print(vtjson['data']['attributes']['last_analysis_stats']) #Debug
                if vtjson['data']['attributes']['last_analysis_stats']['malicious'] > 0:
                    print_results = True #Set the flag to true
                    result += '\n + Flagged as Malicious by: {count} '.format(count=vtjson['data']['attributes']['last_analysis_stats']['malicious'])
                elif vtjson['data']['attributes']['last_analysis_stats']['suspicious'] > 0:
                    print_results = True #Set the flag to true
                    result += '\n + Flagged as Suspicious by: {count} '.format(count=vtjson['data']['attributes']['last_analysis_stats']['suspicious'])
                else:
                    result += '\n + Flagged as Undetected by: {count} '.format(count=vtjson['data']['attributes']['last_analysis_stats']['undetected'])

                #Next, check if the image matches the names
                if include_path_mismatch and path not in vtjson['data']['attributes']['names']:
                    print_results = True #Set the flag to true 
                    result += '\n + The path for {hash} does not match what is reported by VT'.format(hash=hash)
            
        #Error checking from the JSON decoding
        except ValueError:  # includes simplejson.decoder.JSONDecodeError
            print("[ERROR] Issue decoding VirusTotal data ")

    #Now print the result
    if print_results:
        print(result)

    elif vtdata.status_code == 429:
        print('VT Quota Exceeded. Stopping')
        break;
